A Typing is a mathematical model that assigns shape to the data

Example 5 has shape int , "Hello" has shape str

The typing module adds a static type system on otop of python for humans and tools.

**1.Basic Type hints**

In [1]:
#x:int= parameter must be an integer
#x:int -> int means the function returns an integer
def square(x: int) -> int:
    return x * x

print(square(5))  # Output: 25

25


**2. Type aliases**


Its like naming a complex type and reusing it everywhere when we need it.

Lets consider an example with and without type aliases.

without Type Aliases:

In [6]:
# Lets creae a function to store user information
def store_user(user:tuple[int,str,list[tuple[str,int]]])->None:
    """
    user(
        user_id:int,
        user_name:str,
        purchases:list(purchase_name:str,price:int)
    )
    """
    print("Storing user:",user)

store_user((1,"Vamshi",[("laptop",1000),("phone",500)]))

Storing user: (1, 'Vamshi', [('laptop', 1000), ('phone', 500)])


with Type aliases

In [8]:
type UserID=int
type UserName=str
type Purchase=tuple[str,int]
type Purchases=list[Purchase]
type User=tuple[UserID,UserName,Purchases]

def store_user(user:User)->None:
    print("Storing user:",user)

u1:User=(1,"Vamshi",[("laptop",1000),("phone",500)])
store_user(u1)
u2:User=(2,"Moon",[("book",20),("pen",5)])
store_user(u2)

Storing user: (1, 'Vamshi', [('laptop', 1000), ('phone', 500)])
Storing user: (2, 'Moon', [('book', 20), ('pen', 5)])


**New Type (Distinguishing logically different types)**

In [9]:
def get_user(user_id:int)->str:
    return f"User{user_id}"

def product_id(product_id:int)->str:
    return f"Product{product_id}"

print(get_user(1101))
print(product_id(1102))
print(get_user(1102)) # Logical error, but no type error
print(product_id(1101)) # Logical error, but no type error

User1101
Product1102
User1102
Product1101


by this even though we passed int , we might passed product id inplace of user id

In [10]:
from typing import NewType

UserID=NewType("UserID",int)
ProductID=NewType("ProductID",int)

def get_user(user_id:UserID)->str:
    return f"User{user_id}"

def product_id(product_id:ProductID)->str:
    return f"Product{product_id}"

u:UserID=UserID(1101)
p:ProductID=ProductID(1102)

print(get_user(u))
print(product_id(p))
#print(get_user(p)) # Type error
#print(product_id(u)) # Type error

User1101
Product1102


**callable (functions as first class citizens)**

without callable

without callable a function can take any function and just return output without even checking anything


In [13]:
def feeder(fn):
    print(fn())

def greet()->str:
    return "Hell0"

def numer()->int:
    return 42

feeder(greet) # Hell0
feeder(numer) # 42

Hell0
42


There is nothing wrong but with no type checker

In [18]:
from collections.abc import Callable

# this means function calling another function which takes no parameters and returns a string
def feeder(fn:Callable[[],str])->None:
    print(fn())

def greet()->str:
    return "Hell0"

def number()->int:
    return 42

feeder(greet) # Hell0
#feeder(number) # Type error

Hell0


In [ ]:
from collections.abc import Callable

#function which takes two integers and a function which takes two integers and returns an integer
def apply_op(x:int,y:int,op:Callable[[int,int],int])->int:
    return op(x,y)

def divide(a:int,b:int)->int:
    return a/b

def add_str(a:int,b:int)->str:
    return f"{a}+{b}"

print(apply_op(10,5,divide)) # 2.0
#print(apply_op(10,5,add_str)) # Type error


2.0


**Generics and TypeVar**

In [26]:
from collections.abc import Sequence

def first(seq:Sequence)->object:
    return seq[0]

print(first([1,2,3])) # 1 #ok but type checker says object
print(first(["a","b","c"])) # a # ok but again type checker says object

1
a


In [27]:
from typing import TypeVar
from collections.abc import Sequence

T = TypeVar("T")   # T = a placeholder type (like algebra variable X)

def first(seq: Sequence[T]) -> T:
    return seq[0]

# Now the type "flows" correctly
num = first([1, 2, 3])        # num: int
word = first(["a", "b", "c"]) # word: str

print(num + 10)        # OK, int math
print(word.upper())    # OK, str method


11
A


if input is list[int],return is int

if input is list[str], return is str

**Tuples**

A list in Python is usually homogeneous → all elements are of the same type.
Example: list[int] means every element must be an int.

A tuple is often heterogeneous → it can contain elements of different types in fixed positions.
Example: (1, "Alice") → first item is int, second is str.

In [28]:
from typing import Tuple

# Student record: fixed structure
Student = tuple[int, str, float]  # (roll_no, name, GPA)

s1: Student = (101, "Alice", 9.1)   # ✅
s2: Student = (102, "Bob")          # ❌ Missing GPA
s3: Student = ("Charlie", 103, 9.1) # ❌ Wrong order

# Variadic ints: arbitrary length
Scores: tuple[int, ...]
Scores = (10, 20, 30)      # ✅
Scores = (10, "oops", 30)  # ❌ Wrong type

# Empty marker
nothing: tuple[()] = ()    # ✅


In [29]:
s1

(101, 'Alice', 9.1)

In [30]:
s2

(102, 'Bob')

In [31]:
s3

('Charlie', 103, 9.1)

**Any vs object**

Any is like takes any and type checker becomes silent

object is strict one checks everything strictly

In [34]:
from typing import Any

def foo(x: Any) -> None:
    # Type checker will allow *anything*
    print(x + 10)        # ✅ type checker ok (even if x is str)
    x.magic()            # ✅ type checker ok (even if magic() doesn’t exist)
    print(len(x))        # ✅ also ok

foo(5)           # works
foo("hello")     # works at runtime for +10? ❌ -> crashes


15


AttributeError: 'int' object has no attribute 'magic'

In [35]:
def bar(x: object) -> None:
    # Type checker forces you to be explicit
    # x + 10 ❌ error: unsupported operand
    # x.magic() ❌ error: object has no attribute "magic"
    
    # To use it safely, you must check first:
    if isinstance(x, int):
        print(x + 10)  # ✅ ok
    elif isinstance(x, str):
        print(x.upper())  # ✅ ok


In [36]:
from typing import TypedDict

# Define schema
class Student(TypedDict):
    roll_no: int
    name: str
    gpa: float

# ✅ Correct
s1: Student = {"roll_no": 101, "name": "Alice", "gpa": 9.1}

# ❌ Wrong: missing "gpa"
s2: Student = {"roll_no": 102, "name": "Bob"}  

# ❌ Wrong: wrong type (gpa must be float)
s3: Student = {"roll_no": 103, "name": "Charlie", "gpa": "Nine"}  


In [37]:
s1

{'roll_no': 101, 'name': 'Alice', 'gpa': 9.1}